In [1]:
import os
import torch
from torchvision import transforms, models
from PIL import Image
import numpy as np
import torch_directml
from torch.utils.data import Dataset, DataLoader

In [2]:
# Set device
dml = torch_directml.device()

# --- Paths and model ---
root_dir = r'D:\VSC FILES\testtrain\splits_new_v2'
ckpt_path = r'D:\VSC FILES\testtrain\runs\efficientnet_b3_baseline-20251030-102332\best_efficientnet_b3.pth'

# Load checkpoint
ckpt = torch.load(ckpt_path, map_location='cpu')
classes = ckpt['classes']
num_classes = len(classes)

# Model
model = models.efficientnet_b3(weights=None)
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, num_classes)
model.load_state_dict(ckpt['model_state'])
model = model.to(dml)
model.eval()

# Transforms (use eval transforms from training)
mean, std = ckpt['mean'], ckpt['std']
eval_tfms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(ckpt['image_size']),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

C:\Users\JP\AppData\Local\Temp\ipykernel_30224\4159809392.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location='cpu')


In [3]:
class ImageFolderFlat(Dataset):
    def __init__(self, root, classes, transform=None):
        self.samples = []
        self.labels = []
        self.filenames = []
        self.class_to_idx = {c: i for i, c in enumerate(classes)}
        for cls in classes:
            cls_dir = os.path.join(root, cls)
            if not os.path.isdir(cls_dir):
                continue
            # Walk class directory recursively to include images in nested subfolders (e.g. train/val/test)
            for dirpath, _, filenames in os.walk(cls_dir):
                for fn in filenames:
                    if fn.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.webp')):
                        p = os.path.join(dirpath, fn)
                        self.samples.append(p)
                        self.labels.append(self.class_to_idx[cls])
                        self.filenames.append(fn)
        self.transform = transform
        self.classes = classes

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img = Image.open(self.samples[idx]).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, self.labels[idx], self.samples[idx]
        
# Create dataset and dataloader
dataset = ImageFolderFlat(root_dir, classes, transform=eval_tfms)
loader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=0)

In [4]:
bad_images = []

with torch.no_grad():
    for imgs, labels, paths in loader:
        imgs = imgs.to(dml)
        logits = model(imgs)
        probs = torch.softmax(logits, dim=1).cpu().numpy()
        labels = labels.numpy()
        for i in range(len(imgs)):
            top5_idx = probs[i].argsort()[-5:][::-1]
            top5_probs = probs[i][top5_idx]
            top5_classes = [classes[j] for j in top5_idx]
            gt_class = classes[labels[i]]
            if gt_class in top5_classes:
                idx = top5_classes.index(gt_class)
                if top5_probs[idx] < 0.7:
                    bad_images.append(paths[i])
            else:
                bad_images.append(paths[i])

print(f"Images NOT predicted with >=70% confidence in top-5 ({len(bad_images)}):")
for path in bad_images:
    print(path)

Images NOT predicted with >=70% confidence in top-5 (3575):
D:\VSC FILES\testtrain\splits_new_v2\adobong_pusit\test\adobong_pusit_175.jpg
D:\VSC FILES\testtrain\splits_new_v2\adobong_pusit\test\adobong_pusit_2.jpg
D:\VSC FILES\testtrain\splits_new_v2\adobong_pusit\test\adobong_pusit_234.jpg
D:\VSC FILES\testtrain\splits_new_v2\adobong_pusit\test\adobong_pusit_262.jpg
D:\VSC FILES\testtrain\splits_new_v2\adobong_pusit\test\adobong_pusit_264.jpg
D:\VSC FILES\testtrain\splits_new_v2\adobong_pusit\test\adobong_pusit_46.jpg
D:\VSC FILES\testtrain\splits_new_v2\adobong_pusit\test\adobong_pusit_51.jpg
D:\VSC FILES\testtrain\splits_new_v2\adobong_pusit\train\adobong_pusit_107.jpg
D:\VSC FILES\testtrain\splits_new_v2\adobong_pusit\train\adobong_pusit_109.jpg
D:\VSC FILES\testtrain\splits_new_v2\adobong_pusit\train\adobong_pusit_111.jpg
D:\VSC FILES\testtrain\splits_new_v2\adobong_pusit\train\adobong_pusit_138.jpg
D:\VSC FILES\testtrain\splits_new_v2\adobong_pusit\train\adobong_pusit_160.jpg
D:\